In [1]:
import nltk
import unidecode
import pandas as pd
import os, re, string, gzip, itertools
import numpy as np
import tensorflow as tf
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [39]:
!pip install unidecode

In [7]:
!pip3 install keras

In [107]:
ls

BBC News_LSTM.ipynb.html            classfication.ipynb
POC /                               iTerm.app/
bert_uncased_L-12_H-768_A-12_1.tar


In [109]:
df = pd.read_csv("toxic_comments.csv")
df_toxic = df.drop(['id', 'comment_text'], axis=1)
counts = []
categories = list(df_toxic.columns.values)
for i in categories:
    counts.append((i, df_toxic[i].sum()))
df_stats = pd.DataFrame(counts, columns=['category', 'number_of_comments'])
df_stats

,category,number_of_comments
0,toxic,15294
1,severe_toxic,1595
2,obscene,8449
3,threat,478
4,insult,7877
5,identity_hate,1405


In [110]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train, test = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)
X_train = train.comment_text
X_test = test.comment_text
print(X_train.shape)
print(X_test.shape)

(106912,)
(52659,)


In [138]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    print( SVC_pipeline.predict(X_test[:5]))
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing Features
['from vmbkubvmsdccbuffaloedu neil b gandler subject need diode model for pspice organization university at buffalo lines newssoftware vaxvms vnews nntppostinghost ubvmsdccbuffaloedu im designing a circuit with just a silicon diode i dont need to modify any of the parameters but the problem will not accept the following statement model diode d the pspice book i have is terrible i would appreciate any help neil gandler gandler electronics home automation electronic design technology'
 'from rick miller former spook subject alternate legal wiretaps organization just me lines nntppostinghost summary nothing spooky its an executive order tuinstrasignalececlarksonedusoe writes it would be a strong incentive as vesselin points out for more police agencies to go rogue and try to get keys through more efficient but less constitutional means notice what the release said q suppose a law enforcement agency is conducting a wiretap on a drug smuggling ring and intercepts a c

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [40]:
def calc_accuracy_score(clf, train_data, train_target, test_data, test_target, gs = True):
    clf_ = clf.fit(train_data, train_target)
    predicted_test_target = clf_.predict(test_data)
    predicted_train_target = clf_.predict(train_data)
    if True:
        print('For {} classifier accuracy on train data is {}'.format(clf_.steps[-1][-1], accuracy_score(train_target,predicted_train_target)))
        print('For {} classifier accuracy on test data is {}'.format(clf_.steps[-1][-1], accuracy_score(test_target, predicted_test_target)))
    else:
        print('For {} classifier accuracy on train data is {}'.format(clf_, accuracy_score(train_target,predicted_train_target)))
        print('For {} classifier accuracy on test data is {}'.format(clf_, accuracy_score(test_target, predicted_test_target)))
        

In [41]:

def clean_text(text):
    text = re.sub(pattern=r'<.*?>', repl=' ', string=text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    text="".join([i for i in text if i not in string.punctuation])
    text = unidecode.unidecode(text)
    text = text.strip()
  
    return text



In [88]:
def CleanText(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [117]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words =set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/satyammishra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [139]:
categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']

In [140]:
twenty_train = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

In [119]:
twenty_train.target

array([7, 4, 4, ..., 3, 1, 8])

In [141]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42))])
text_clf_stop = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])

NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])


classfiers = [text_clf, text_clf_svm, text_clf_stop, SVC_pipeline, LogReg_pipeline, NB_pipeline]
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}

In [142]:
my_dict = dict(zip(twenty_train.data , twenty_train.target))
train= pd.DataFrame.from_dict(my_dict, orient= 'index')
train.reset_index(inplace=True)
train.columns = ['Features', 'Target']
train.head()

,Features,Target
0,From: sd345@city.ac.uk (Michael Collier)\nSubj...,1
1,From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\...,1
2,From: djohnson@cs.ucsd.edu (Darin Johnson)\nSu...,3
3,From: s0612596@let.rug.nl (M.M. Zwart)\nSubjec...,3
4,From: stanly@grok11.columbiasc.ncr.com (stanly...,3


In [143]:
my_dict = dict(zip(twenty_test.data , twenty_test.target))
test= pd.DataFrame.from_dict(my_dict, orient= 'index')
test.reset_index(inplace=True)
test.columns = ['Features', 'Target']
test.head()

,Features,Target
0,From: brian@ucsd.edu (Brian Kantor)\nSubject: ...,2
1,From: rind@enterprise.bih.harvard.edu (David R...,2
2,From: adwright@iastate.edu ()\nSubject: Re: ce...,2
3,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,0
4,From: jhpb@sarto.budd-lake.nj.us (Joseph H. Bu...,3


In [123]:
train.Features = train['Features'].apply(clean_text)
test.Features = test['Features'].apply(clean_text)

In [124]:
train.Features = train['Features'].apply(CleanText)
test.Features = test['Features'].apply(CleanText)

In [125]:
one_hot = pd.get_dummies(train['Target'])
# Drop column B as it is now encoded
train = train.drop('Target',axis = 1)
# Join the encoded df
train = train.join(one_hot)


In [126]:
one_hot = pd.get_dummies(test['Target'])
# Drop column B as it is now encoded
test = test.drop('Target',axis = 1)
# Join the encoded df
test = test.join(one_hot)

In [127]:
categories

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [128]:
counts = []
categories = list(train.columns.values)
# for i in categories:
#     counts.append((i, df[i].sum()))
# df_stats = pd.DataFrame(counts, columns=['category', 'number_of_comments'])
# df_stats

In [129]:
# train, test = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)


In [144]:
X_train = train.Features
X_test  = test.Features

In [136]:
X_test[:10]

0    from vmbkubvmsdccbuffaloedu neil b gandler sub...
1    from rick miller subject xface organization ju...
2    from mathew subject re strong weak atheism org...
3    from bakkencsarizonaedu dave bakken subject re...
4    from liveseysolntzewpdsgicom jon livesey subje...
5    from banschbachvmsocomokstateedu subject re ca...
6    from petchgvggvgtekcom chuck subject daily ver...
7    from fortmannsuperbowlundacza paul fortmann pg...
8    from kartikhlscom kartik chandrasekhar subject...
9    from tmcspartanacbrockuca tim ciceran subject ...
Name: Features, dtype: object

In [145]:
count =0
for classfier in classfiers:
    count+=1
    print("Model : {} ".format(count))
    calc_accuracy_score(classfier, X_train , twenty_train.target, X_test, twenty_test.target)
    print("\n")

Model : 1 
For MultinomialNB() classifier accuracy on train data is 0.9636685866194062
For MultinomialNB() classifier accuracy on test data is 0.8348868175765646


Model : 2 
For SGDClassifier(alpha=0.001, random_state=42) classifier accuracy on train data is 0.997341603898981
For SGDClassifier(alpha=0.001, random_state=42) classifier accuracy on test data is 0.9114513981358189


Model : 3 
For MultinomialNB() classifier accuracy on train data is 0.9884802835622508
For MultinomialNB() classifier accuracy on test data is 0.8894806924101198


Model : 4 
For OneVsRestClassifier(estimator=LinearSVC(), n_jobs=1) classifier accuracy on train data is 1.0
For OneVsRestClassifier(estimator=LinearSVC(), n_jobs=1) classifier accuracy on test data is 0.9260985352862849


Model : 5 
For OneVsRestClassifier(estimator=LogisticRegression(solver='sag'), n_jobs=1) classifier accuracy on train data is 0.9924678777137793
For OneVsRestClassifier(estimator=LogisticRegression(solver='sag'), n_jobs=1) classif

In [66]:
test

,Features,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,from vmbkubvmsdccbuffaloedu neil b gandler\nsu...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,from rick miller \nsubject xface\norganizatio...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,from mathew \nsubject re strong weak atheism...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,from bakkencsarizonaedu dave bakken\nsubject r...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,from liveseysolntzewpdsgicom jon livesey\nsubj...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7527,from richmondspiffprincetonedu stupendous man\...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7528,from smytonjmurrallegedu jim smyton\nsubject r...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7529,from hhendersonvaxclarkuedu\nsubject re game l...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7530,from bzamutarlgutaedu \nsubject intel chmos d...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
categories

['Features',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19]

In [101]:
for category in categories[1:5]:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print(SVC_pipeline.predict(X_test[:100]))
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing 0
[0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Test accuracy is 0.9799522039298991
... Processing 1
[0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Test accuracy is 0.9731810939989378
... Processing 2
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Test accuracy is 0.9709240573552841
... Processing 3
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Test acc

In [37]:
NB_pipeline.predict(np.array([X_test[1]]))

array([0], dtype=uint8)

In [92]:
categories

['Features', 'Target']

In [ ]:

for i in range(10):
    prediction = NB_pipeline.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_narrative.iloc[i][:50], "...")
    print('Actual label:' + test_product.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

In [ ]:
# count =0
for classfier in classfiers:
    count+=1
    print("Model : {} ".format(count))
    calc_accuracy_score(classfier, X_train, y_train, X_test, y_test)
    print("\n")
    

Model : 1 
For MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) classifier accuracy on train data is 0.9261213720316622
For MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) classifier accuracy on test data is 0.8299410819496519


Model : 2 
For SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False) classifier accuracy on train data is 0.9740105540897097
For SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=

In [ ]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)

gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)
gs_clf.best_score_
gs_clf.best_params_
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
gs_clf_svm.best_score_
gs_clf_svm.best_params_

{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [ ]:
# count =0
# for classfier in [gs_clf, gs_clf_svm]:
#     count+=1
#     print("Model : {} ".format(count))
#     calc_accuracy_score(classfier, twenty_train.data, twenty_train.target, twenty_test.data, twenty_test.target, gs=False)
#     print("\n")

In [ ]:
parameters = {
    "estimator__C": [1,2,4,8],
    "estimator__kernel": ["poly","rbf"],
    "estimator__degree":[1, 2, 3, 4],
}
gs_clf = GridSearchCV(SVC_pipeline, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)




In [ ]:
gs_clf.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__tfidf', 'estimator__clf', 'estimator__tfidf__analyzer', 'estimator__tfidf__binary', 'estimator__tfidf__decode_error', 'estimator__tfidf__dtype', 'estimator__tfidf__encoding', 'estimator__tfidf__input', 'estimator__tfidf__lowercase', 'estimator__tfidf__max_df', 'estimator__tfidf__max_features', 'estimator__tfidf__min_df', 'estimator__tfidf__ngram_range', 'estimator__tfidf__norm', 'estimator__tfidf__preprocessor', 'estimator__tfidf__smooth_idf', 'estimator__tfidf__stop_words', 'estimator__tfidf__strip_accents', 'estimator__tfidf__sublinear_tf', 'estimator__tfidf__token_pattern', 'estimator__tfidf__tokenizer', 'estimator__tfidf__use_idf', 'estimator__tfidf__vocabulary', 'estimator__clf__estimator__C', 'estimator__clf__estimator__class_weight', 'estimator__clf__estimator__dual', 'estimator__clf__estimator__fit_intercept', 'estimator__clf__estimator__intercept_scaling', 'estimator__clf

In [ ]:
df['Product'].value_counts()

In [ ]:
# Split data into train and test
train_size = int(len(df) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(df) - train_size))

In [ ]:
train_narrative = twen
train_product = df['Product'][:train_size]

test_narrative = df['Consumer complaint narrative'][train_size:]
test_product = df['Product'][train_size:]

In [ ]:
train_narrative = twenty_train.data
train_product = twenty_train.target

test_narrative = twenty_test.data
test_product = twenty_test.target

In [ ]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_narrative) # only fit on train
x_train = tokenize.texts_to_matrix(train_narrative)
x_test = tokenize.texts_to_matrix(test_narrative)

In [ ]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_product)
y_train = encoder.transform(train_product)
y_test = encoder.transform(test_product)

In [ ]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (11314, 1000)
x_test shape: (7532, 1000)
y_train shape: (11314, 20)
y_test shape: (7532, 20)


In [ ]:
batch_size = 32
epochs = 5

In [ ]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

2021-08-13 23:34:45.057190: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

2021-08-13 23:34:49.597499: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
319/319 [==============================] - 19s 7ms/step - loss: 2.2704 - accuracy: 0.3393 - val_loss: 0.9573 - val_accuracy: 0.7314
Epoch 2/5
319/319 [==============================] - 1s 4ms/step - loss: 0.8193 - accuracy: 0.7710 - val_loss: 0.7684 - val_accuracy: 0.7721
Epoch 3/5
319/319 [==============================] - 1s 4ms/step - loss: 0.5417 - accuracy: 0.8453 - val_loss: 0.7313 - val_accuracy: 0.7836
Epoch 4/5
319/319 [==============================] - 1s 4ms/step - loss: 0.4035 - accuracy: 0.8877 - val_loss: 0.7382 - val_accuracy: 0.7792
Epoch 5/5
319/319 [==============================] - 1s 4ms/step - loss: 0.3181 - accuracy: 0.9146 - val_loss: 0.7332 - val_accuracy: 0.7959


In [ ]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

236/236 [==============================] - 1s 2ms/step - loss: 1.2025 - accuracy: 0.6668
Test score: 1.2024712562561035
Test accuracy: 0.666755199432373


In [ ]:
# # Here's how to generate a prediction on individual examples
# text_labels = encoder.classes_ 


In [ ]:
text = 'Hello 123   <br> hghjh</br> completed implementation dèèp lèarning ánd cömputer vísíön satyam@gmail.com , p>>..>>>#9 YUJ hsddu gh ssd [[23/23/344]] https:www.google.com js'

In [ ]:
clean_text(text)

'hello      hghjh  completed implementation deep learning and computer vision satyamgmailcom  p yuj hsddu gh ssd   js'